In [27]:
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import pickle
import unicodedata
import codecs
import re
import random
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import itertools

In [2]:
file = open('D:\Spring22 - UIC\SP22 Notebooks\IDS 576\Project\yahoo data\manner-v2.0\manner-2.xml', encoding="UTF-8")

In [3]:
data=file.read()

In [4]:
bs_data = BeautifulSoup(data, 'xml') 

In [5]:
b_unique = bs_data.find_all('vespaadd') 

In [6]:
l_uri=[]
for ques in b_unique:
    l_uri.append(ques.find_all('uri'))
l_uri[0][0].get_text()

'56910'

In [7]:
len(l_uri)

142627

In [8]:
l_sbj=[]
for ques in b_unique:
    l_sbj.append(ques.find_all('subject'))
l_sbj[0][0].get_text()

'How to handle a 1.5 year old when hitting?'

In [9]:
len(l_sbj)

142627

In [10]:
l_con=[]
for ques in b_unique:
    l_con.append(ques.find_all('content'))
l_con[0][0].get_text()

'Hi,Our 17 month old has started hitting, we don\'t believe in physical discipline. Any tips other parents would recommend?We currently tell him "no hitting", "hitting hurts", and he kind of gets it. But sometimes clearly that doesn\'t help much...particularly when he\'s frustrated.Any tips?'

In [11]:
type(l_sbj[0][0])

bs4.element.Tag

In [12]:
l_bans=[]
for ques in b_unique:
    l_bans.append(ques.find_all('bestanswer'))
l_bans[0][0].get_text()

'you\'re on the right track in understanding that he\'s frustrated. my viewpoint is that the frustration really comes about in not being able to communicate what he wants to person being hit. the hitting action then becomes his only recourse because he sees that it gets your attention.something my daughter\'s preschool does is a time-with versus a time-out and i found better results with my 3 year old. the first thing discussed is having her empathize with the other person by going beyong the "hitting hurts" and asking her "how do you think hitting makes her feel?" the second thing we do is get her to communicate her frustration is words rather than action, helping her along the way with getting the words right.'

In [13]:
len(l_bans)

142627

In [14]:
df = pd.DataFrame([l_uri,l_sbj,l_con,l_bans])

In [15]:
df_new=df.T
df_new.columns=['Q_ID','Q_Subject','Q_Content','Best Answer']
# df_new.head(10)

,Q_ID,Q_Subject,Q_Content,Best Answer
0,[[56910]],[[How to handle a 1.5 year old when hitting?]],"[[Hi,Our 17 month old has started hitting, we ...",[[you're on the right track in understanding t...
1,[[109662]],[[How can I avoid getting sick in China?]],[[I'm planning a trip to China. I'm an adventu...,[[Number one cause of digestive complaints is ...
2,[[4759]],[[How do male penguins survive without eating ...,[[How do male penguins survive without eating ...,[[Male penguins don't eat for 60 days. The fem...
3,[[87264]],[[How do I remove candle wax from a polar flee...,[[Spilled some wax on a brand new fleece jacke...,[[I've heard the best thing to do is to try to...
4,[[99220]],[[How do I find an out of print book?]],[[When I was a kid I remember seeing a book th...,[[There are several websites that you can find...
5,[[22376]],[[How to clean ickiness off a plastic shower c...,[[I am having a plumbing problem that means th...,[[How about an ammonia spray that you can coat...
6,[[56497]],[[How do I take my money out of 401k without p...,"[[If I change a job, and roll my 401k into an ...",[[You can withdraw the money for childrens hig...
7,[[81535]],[[How to make perfect ribs]],[[how to make perfect ribs]],"[[I'm from Kansas, and KC BBQ is of course my ..."
8,[[37733]],[[How do I get rid of my California ant proble...,"[[Ever since I moved to California, I've been ...","[[In a similar situation, after trying everyth..."
9,[[8656]],[[How to remove tree sap from car?]],"[[Hi,I have some tree sap on my car, both olde...",[[I have had good luck with commercial product...


In [16]:
df_new['Q_ID']=df_new['Q_ID'].apply(lambda x: x[0].get_text())
df_new['Q_Subject']=df_new['Q_Subject'].apply(lambda x: x[0].get_text())
df_new['Q_Content']=df_new['Q_Content'].apply(lambda x: "" if len(x) == 0 else x[0].get_text())
df_new['Best Answer']=df_new['Best Answer'].apply(lambda x: x[0].get_text())

In [18]:
df_new.shape

(142627, 4)

In [58]:
# f=open('yahoo_data.pkl','wb')
# pickle.dump(df_new,f)
f=open('yahoo_data.pkl','rb')
df_new=pickle.load(f)

In [62]:
df_conv = df_new[['Q_Subject','Best Answer']].rename(columns={'Q_Subject':'question','Best Answer':'answer'})
df_conv=df_conv.reset_index().drop(columns='index')

In [63]:
len_col={}
for i in df_conv.columns:
    max=0
    len_list=[]
    for j in range(df_conv.shape[0]): 
        len_list.append(len(df_conv[i][j].split()))
        if len(df_conv[i][j].split()) > max:
            max = len(df_conv[i][j].split())
    len_col[i] = len_list
    print(f'Max length of column {i} is {max}.')     

Max length of column question is 32.
Max length of column answer is 890.


In [69]:
pd.DataFrame(len_col).describe()

,question,answer
count,142626.000000,142626.000000
mean,11.036480,79.488747
std,4.369146,97.778742
min,4.000000,1.000000
25%,8.000000,23.000000
50%,10.000000,48.000000
75%,13.000000,95.000000
max,32.000000,890.000000


In [99]:
# for j in range(df_conv.shape[0]): 
#     if len(df_conv['question'][j+1].split()) > 8:
#         df_conv.drop(j,inplace=True)
# print(f'Size of Dataset is {df_conv.shape[0]}.') 

Size of Dataset q is 47053.


In [111]:
for j in range(df_conv.shape[0]): 
    if len(df_conv['answer'][j].split()) > 23:
        df_conv.drop(j,inplace=True)
print(f'Size of Dataset is {df_conv.shape[0]}.') 

Size of Dataset is 35854.


In [115]:
df_conv=df_conv.reset_index().drop(columns='index')
# df_conv.head()

,question,answer
0,How to get rid of water stain in a stall shower?,Sounds like hard water deposits. I find that L...
1,How do you remove permanent marker on applianc...,Just use hand sanitizer to get the stain out. ...
2,How to fix vertical blinds?,Click on the link and follow the steps.
3,How to clean window screens?,Nylon covered sponges are great for cleaning w...
4,How to get rid of rodents?,"Use glue traps for small mice, and spring trap..."


In [2]:
# f=open('yahoo_data_subset.pkl','wb')
# pickle.dump(df_conv,f)
f=open('yahoo_data_subset.pkl','rb')
df_subset=pickle.load(f)

In [20]:
lines=[]
for i in range(len(df_subset)):
    lines.append('\t'.join(df_subset.iloc[i]))

contractions = {
    "don't": "do not",
    "didn't": "did not",
    "isn't": "is not",
    "can't": "cannot",
    "wasn't": "was not",
    "won't": "will not",
    "doesn't": "does not",
    "couldn't": "could not",
    "haven't": "have not",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "hasn't": "has not",
    "aren't": "are not",
    "weren't": "were not"
}

def uncontract(s):
    return ' '.join(contractions[c] if c in contractions else c for c in s.split())

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', uncontract(s))
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r'[^\w\s]','',s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [19]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [7]:
# from torchtext.vocab import GloVe
# embedding_glove = GloVe(name='6B', dim=300)

100%|███████████████████████████████████████████████████████████████████████▉| 399999/400000 [00:43<00:00, 9261.13it/s]


In [10]:
eng_glove_300_dim = pd.read_csv(".vector_cache/glove.6B.300d.txt", sep = " ", index_col=0, header=None, quoting=3)

In [21]:
gloveindex = set(eng_glove_300_dim.index)
pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
print(f"{len(pairs)} sentences before removing those with words missing in GloVe")
pairs = [pair for pair in pairs if len(set(pair[0].split()) - gloveindex) == 0 & len(set(pair[1].split()) - gloveindex) == 0]
print(f"{len(pairs)} sentences after removing those with words missing in GloVe")
voc = Voc('Yahoo_subdata')
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("Counted words:", voc.num_words)

35854 sentences before removing those with words missing in GloVe
31034 sentences after removing those with words missing in GloVe
Counted words: 37957


## Constructing the Training Data

In [33]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


In [46]:
# # Example for validation
# small_batch_size = 7
# batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
# input_variable, lengths, target_variable, mask, max_target_len = batches

# print("input_variable:", input_variable)
# print("lengths:", lengths)
# print("target_variable:", target_variable)
# print("mask:", mask)
# print("max_target_len:", max_target_len)

## Building the Encoder RNN

In [61]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

## Constructing the Attention

In [47]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

## Constructing the Decoder

In [48]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## Defining Loss

In [51]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Training 

In [55]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

## Training Iterations

In [56]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## Decoding the input

In [57]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

## Evaluation

In [59]:
def evaluate(encoder, decoder, searcher, voc, sentence):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

## Training the Model

In [1]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...


NameError: name 'nn' is not defined